In [16]:
# Import libraries
from math import atan2, cos, sin, sqrt, pi
import numpy as np
import cv2 as cv2
import os
import glob
import time
import pandas as pd
#from scipy.signal import butter, lfilter
#from scipy import signal
#from scipy.signal import find_peaks, resample_poly

# from joblib import Parallel, delayed
# import multiprocessing
# from ipywidgets import interactive, widgets
# from skimage.registration import phase_cross_correlation
# from skimage.registration._phase_cross_correlation import _upsampled_dft
# from scipy.ndimage import fourier_shift
from matplotlib.image import imread
import matplotlib.pyplot as plt 
from PIL import Image
import PyCBD.pipelines
from PyCBD.pipelines import CBDPipeline
from PyCBD.checkerboard_detection.checkerboard_detector import CheckerboardDetector
import matplotlib.transforms as mtransforms


In [17]:
im_dir1 = r"D:\FST_972_FF_Sphere\Calibration\2100K_Checkerboard_FF"
im_dir2 = r"D:\FST_972_FF_Sphere\Calibration\480K_CheckerBoard_ff_C001H001S0001"
files1  = sorted(glob.glob(os.path.join(im_dir1, "*.tif"))) 
files2  = sorted(glob.glob(os.path.join(im_dir2, "*.tif"))) 
#glob.glob lists all files that start with im_dir and end with tif
#output_dir1= r"D:\FST_972_FF_Sphere\FST_972_Run\2100K_FF_Run\Masked_images"
#os.mkdir(output_dir1)
#output_dir2=r"D:\FST_972_FF_Sphere\FST_972_Run\480K_FF_Run\Masked_images"
#os.mkdir(output_dir2)

## Testing Open CV's checkerboard detection 

In [ ]:
# Set the checkerboard size (internal corners, not squares)
board_size = (8, 3)  # Adjust based on your checkerboard

# Load a single image (change path accordingly)
image_path = r"D:\FST_972_FF_Sphere\Calibration\2100K_Processed_Images\Processed_Partition 1000061.tif"
#image = cv2.imread(image_path)
img   = Image.open(image_path)
image =  np.array(img)

# Convert 16-bit to 8-bit if needed
if image.dtype != np.uint8:
    gray = (image/16).astype(np.uint8)

# Detect the checkerboard corners
ret, corners = cv2.findChessboardCorners(gray, board_size, None)

# If corners are found, refine and draw them
if ret:
    corners_refined = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), 
                                       criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
    cv2.drawChessboardCorners(image, board_size, corners_refined, ret)

    print(f"✅ Checkerboard detected with {len(corners)} corners.")
else:
    print("❌ Checkerboard NOT detected. Try adjusting lighting or focus.")

# Show the image with detected corners

cv2.namedWindow("Checkerboard Detection", cv2.WINDOW_NORMAL)  # Allows window resizing
cv2.resizeWindow("Checkerboard Detection", 800, 600)  # Set to 800x600 (adjust as needed)
cv2.imshow("Checkerboard Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
# cv2.imshow("Checkerboard Detection", image)
# cv2.waitKey(0) #keeps displayed image open for (duration)
#cv2.destroyAllWindows()

In [23]:
# Load the image
#image_path = "/mnt/data/image.png"  # Update if needed
img = Image.open(image_path)
image = np.array(img)

# Normalize properly (adaptive scaling based on actual pixel values)
if image.dtype == np.uint16:
    min_val = np.min(image)
    max_val = np.max(image)
    print(f"Min Pixel Value: {min_val}, Max Pixel Value: {max_val}")  # Debugging

    # Scale the 16-bit image to 8-bit properly
    gray = ((image - min_val) / (max_val - min_val) * 255).astype(np.uint8)
else:
    gray = image.astype(np.uint8)

# Display the converted image
cv2.imshow("Converted Image", gray)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [20]:
gray.shape

(840, 1024)

In [ ]:
# Define input and output directories
input_folder = r"D:\FST_972_FF_Sphere\Calibration\2100K_Checkerboard_FF"
output_folder = r"D:\FST_972_FF_Sphere\Calibration\2100K_Processed_Images_test"

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Initialize detection pipeline
detection_pipeline = CBDPipeline()

# Arrays to store detected points
detected_data = []  # Store (image_name, board_uv, board_xy)
i=0

# Process each image in the folder
for file in sorted(os.listdir(input_folder)):
    if file.endswith(".tif"):  # Process only TIFF images
        fname = os.path.join(input_folder, file)
        img = Image.open(fname)
        image = np.array(img)
        
        # Convert 16-bit to 8-bit if needed
        if image.dtype == np.uint16:
            image = (image / 16).astype(np.uint8)
        
        # Detect checkerboard
        result, board_uv, board_xy = detection_pipeline.detect_checkerboard(image)
        
        if result:
            detected_data.append((i, board_uv, board_xy))
            
            # Plot result
            fig, ax = plt.subplots()
            ax.imshow(image, cmap='gray')
            ax.plot(board_uv[:, 0], board_uv[:, 1], 'r-o', markeredgecolor='k')
            
            # Add annotations
            trans_offset = mtransforms.offset_copy(ax.transData, fig=fig, x=-0.4, y=-0.20, units='inches')
            ax.text(board_uv[0, 0], board_uv[0, 1], f'({int(board_xy[0, 0])}, {int(board_xy[0, 1])})',
                    color="red", transform=trans_offset)
            trans_offset = mtransforms.offset_copy(ax.transData, fig=fig, x=0.05, y=0.05, units='inches')
            ax.text(board_uv[-1, 0], board_uv[-1, 1], f'({int(board_xy[-1, 0])}, {int(board_xy[-1, 1])})',
                    color="red", transform=trans_offset)
            
            plt.title(f"Detection result: {file}")
            plt.axis('off')
            
            # Save the plot
            output_fname = os.path.join(output_folder, f"Processed_{file}")
            plt.savefig(output_fname, bbox_inches='tight', dpi=300)
            plt.close(fig)
        i+=1

# Save detected data
np.save(os.path.join(output_folder, "detected_checkerboard_data.npy"), detected_data)

print("Processing completed. All images are saved in", output_folder)